In [ ]:

import cv2
import os
import numpy as np
import matplotlib.pyplot as plt

directorio = "/home/maucasco/Documents/maestria/proyecto_grado/pujc-advocato-filter-project-mngr/assets"
def eliminar_sombras(image):


# Ecualizar el histograma
    kernel = np.ones((5, 5), np.uint8)
    tophat = cv2.morphologyEx(image, cv2.MORPH_TOPHAT, kernel)

    # Mejorar el contraste después de la transformación 'top-hat'
    result = cv2.add(image, tophat)
    print(result)
    return result


def eliminar_texto(imagen, x_inicio, y_inicio, x_fin, y_fin):
    mascara = np.zeros_like(imagen[:, :, 0])
    mascara[y_inicio:y_fin, x_inicio:x_fin] = 255
    dilatada = cv2.dilate(mascara, (1,1), iterations = 2)  # Dilatar para cubrir más región del texto
    return cv2.inpaint(imagen, dilatada, inpaintRadius=3, flags=cv2.INPAINT_TELEA)

def mejorar_contraste(imagen):
    # Ecualización del histograma
    ecualizada = cv2.equalizeHist(imagen)
    
    # Ajuste de contraste y brillo
    contraste = 1.5  # Factor de contraste (mayor que 1 aumenta el contraste)
    brillo = 30      # Ajuste de brillo
    mejorada = cv2.convertScaleAbs(ecualizada, alpha=contraste, beta=brillo)
    
    # Filtro bilateral para suavizar el ruido manteniendo los bordes
    filtrada = cv2.bilateralFilter(mejorada, 9, 75, 75)
    
    return filtrada

def segmentar_imagen(imagen_sin_texto):
   
   
    
    # Eliminar sombras
    imagen_sin_texto= eliminar_sombras(imagen_sin_texto)
    imagen_gris = cv2.cvtColor(imagen_sin_texto, cv2.COLOR_BGR2GRAY)
    
    # Usar thresholding de Otsu
    _, imagen_segmentada = cv2.threshold(imagen_gris, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
    
    # Detección de contornos y conservar solo el contorno más grande
    contours, _ = cv2.findContours(imagen_segmentada, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Filtrar contornos por área
    min_area = 3000  # Puedes ajustar este valor
    contours = [c for c in contours if cv2.contourArea(c) > min_area]
    
    mask = np.zeros_like(imagen_gris)
    if contours:
        # Ordenar contornos y dibujar el más grande
        contours = sorted(contours, key=cv2.contourArea, reverse=True)
        cv2.drawContours(mask, [contours[0]], -1, (255), thickness=cv2.FILLED)
    
    aguacate = cv2.bitwise_and(imagen_gris, imagen_gris, mask=mask)
    
    # Mejorar el contraste y brillo del aguacate
    aguacate_resaltado = mejorar_contraste(aguacate)
    
    return aguacate_resaltado

# Carga y preprocesamiento
def redimensionar(image):
   original_height, original_width = image.shape[:2]
   desired_width = 100

# Calcula el factor de escala y el alto deseado
   scale_factor = desired_width / original_width
   desired_height = 130

# Redimensiona la imagen proporcionalmente
   resized_img = cv2.resize(image, (desired_width, desired_height))


   return resized_img

def procesar_imagen(ruta_imagen,nombre):
    imagen = cv2.imread(ruta_imagen)[100:, :]

    imagen_sin_texto=[]
    print("Type:",type(imagen))
    print("Shape of Image:", imagen.shape)
    print('Total Number of pixels:', imagen.size)
    print("Image data type:",imagen.dtype)
        # print("Pixel Values:\n", img)
    print("Dimension:", imagen.ndim)


    if imagen.size>1000000:
        imagen_sin_texto = eliminar_texto(imagen, 0, 0, 3000, 250)
    elif imagen.size>10000000:
        imagen_sin_texto = eliminar_texto(imagen, 0, 0, 3000, 1000)
    else:
        imagen_sin_texto = eliminar_texto(imagen, 0, 0, 3000, 500)    
        
    aguacate_solo = segmentar_imagen(imagen_sin_texto)
    redimenciada = redimensionar(aguacate_solo)
    nuevaImga=directorio+'/process/fil_'+nombre
    print(nuevaImga)
    cv2.imwrite(nuevaImga, redimenciada)
    return imagen, imagen_sin_texto, aguacate_solo,redimenciada


def mostrar_imagenes(titulos, imagenes):
    count = len(titulos)
    for i in range(count):
        plt.subplot(1, count, i + 1)
        plt.title(titulos[i])
        plt.imshow(imagenes[i], cmap='gray')
    plt.show()

    


In [ ]:

archivos = os.listdir(directorio)
for archivo in archivos:
    if archivo.endswith(".jpg"):
        ruta_imagen = os.path.join(directorio, archivo)
        titulos = ["Ori", "SinTex","Sinsomb", archivo]
        imagenes = procesar_imagen(ruta_imagen,archivo)
        mostrar_imagenes(titulos, imagenes)
     

In [ ]:
# Calcular histograma
def mostrar_histograma(img):
    histogram = cv2.calcHist([img], [0], None, [256], [0,256])

    # Graficar histograma
    plt.figure()
    plt.title("Histograma de Intensidad")
    plt.xlabel("Valor de pixel")
    plt.ylabel("Número de píxeles")
    plt.plot(histogram)
    plt.xlim([0, 256])
    plt.show()

In [ ]:
import cv2
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt


def mostrar_densidad(pixels,archivo):

    # Crear el gráfico de densidad
    sns.set(style="whitegrid")
    plt.figure(figsize=(10, 6))
    sns.kdeplot(pixels, bw_adjust=0.5, shade=True)

    plt.title('Gráfico de Densidad de Valores de Píxeles'+archivo)
    plt.xlabel('Valor del Píxel')
    plt.ylabel('Densidad')
    plt.show()

In [ ]:

def createDataset():
    archivos = os.listdir(directorio+'/process')
    print(archivos)
    data = []
    labels = []
    dimensiones = []
    for archivo in archivos:
        if archivo.endswith(".jpg"):
            ruta_imagen = os.path.join(directorio+'/process/', archivo)
            print(ruta_imagen)
            img = cv2.imread(ruta_imagen)
            #mostrar_histograma(img)
            if img is not None:
                print(directorio+'/process'+archivo)
                flattened = img.flatten()
                data.append(flattened)
                mostrar_densidad(flattened,archivo)
                dimensiones.append(img.shape)
                # Aquí necesitas una forma de obtener la etiqueta para cada imagen
                # Ejemplo: si el nombre del archivo indica la clase
                if "sano" in archivo:
                    labels.append(0)
                else:
                    labels.append(1)
    return data , labels, dimensiones

In [ ]:
import pandas as pd
import seaborn as sns

data , labels, dimensiones =createDataset()



X = np.array(data)
dims = np.array(dimensiones)
df_data = pd.DataFrame(X)

# Agregar columnas para dimensiones
df_data['height'] = dims[:, 0]
df_data['width'] = dims[:, 1]
# Guardar el DataFrame como CSV (opcional)
df_data.to_csv(os.path.join(directorio, 'dataset_imagenes.csv'), index=False)

In [ ]:
!pip install tensorflow

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam

# Definir el modelo
model = Sequential()

# Añadir una capa convolucional con 32 filtros, un kernel de 3x3, activación ReLU y entrada especificada
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(100,100, 1))) # Asegúrate de que input_shape coincida con las dimensiones de tus imágenes
model.add(MaxPooling2D((2, 2)))

# Añadir más capas convolucionales y de pooling según sea necesario
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

# Aplanar el resultado para alimentar una DNN
model.add(Flatten())

# Añadir capas densas (fully connected)
model.add(Dense(128, activation='relu'))

# Añadir Dropout para evitar overfitting
model.add(Dropout(0.5))

# Capa de salida con activación softmax para clasificación multiclase
model.add(Dense(10, activation='softmax')) # Cambia el número de neuronas a la cantidad de tus clases

# Compilar el modelo
model.compile(optimizer=Adam(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Ver la arquitectura del modelo
model.summary()

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Crear generadores de datos de imagen
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

# Suponiendo que tus datos están en un directorio llamado 'data' con subdirectorios 'train' y 'test'
train_generator = train_datagen.flow_from_directory(
    'assets/traint',
    target_size=(100, 100),
    color_mode='grayscale',  # o 'rgb' si tus imágenes son a color
    batch_size=32,
    class_mode='sparse')

test_generator = test_datagen.flow_from_directory(
    '/home/maucasco/Documents/maestria/proyecto_grado/pujc-advocato-filter-project-mngr/assets/test',
    target_size=(100, 100),
    color_mode='grayscale',  # o 'rgb' si tus imágenes son a color
    batch_size=32,
    class_mode='sparse')

# Entrenar el modelo
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=test_generator,
    validation_steps=test_generator.samples // test_generator.batch_size,
    epochs=5)